## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-16-38-57 +0000,nypost,"Nepal sets elections, names interim prime mini...",https://nypost.com/2025/09/13/world-news/nepal...
1,2025-09-13-16-34-15 +0000,nypost,"Knicks center Mitchell Robinson slams critics,...",https://nypost.com/2025/09/13/us-news/knicks-c...
2,2025-09-13-16-19-55 +0000,nypost,Charlie Kirk’s alleged killer Tyler Robinson f...,https://nypost.com/2025/09/13/us-news/yearbook...
3,2025-09-13-16-19-41 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/charlie-...
4,2025-09-13-16-19-17 +0000,cbc,China launches anti-discrimination investigati...,https://www.cbc.ca/news/business/china-discrim...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,kirk,46
19,charlie,41
101,trump,23
175,shooting,13
220,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
117,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,134
122,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,131
96,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...,130
139,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,130
183,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,130


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
117,134,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...
13,55,2025-09-13-15-34-13 +0000,nypost,Trump ready to hit Moscow with ‘major’ sanctio...,https://nypost.com/2025/09/13/us-news/trump-re...
96,43,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...
16,35,2025-09-13-15-26-15 +0000,nypost,Zohran Mamdani PAC’s top donors worked for fir...,https://nypost.com/2025/09/13/us-news/top-dono...
29,35,2025-09-13-14-43-23 +0000,nypost,6 more officers fired after woman warned cops ...,https://nypost.com/2025/09/13/us-news/6-more-o...
244,33,2025-09-12-17-08-00 +0000,bbc,Anti-apartheid hero's new inquest should lead ...,https://www.bbc.com/news/articles/c931n9eelpeo...
102,29,2025-09-13-08-42-17 +0000,bbc,Colombian court rules Meta was wrong to bar po...,https://www.bbc.com/news/articles/cp8wlxwy1exo...
216,29,2025-09-12-18-40-00 +0000,wsj,How Israel Used Ballistic Missiles From the Re...,https://www.wsj.com/world/middle-east/how-isra...
22,28,2025-09-13-15-08-00 +0000,wsj,Fed Reserve governor Lisa Cook described one o...,https://www.wsj.com/economy/central-banking/do...
243,28,2025-09-12-17-08-33 +0000,nypost,"Roger Marshall, Raphael Warnock push plan to b...",https://nypost.com/2025/09/12/us-news/sens-mar...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
